#### Import neccesary libraries

In [25]:
import os
import pandas as pd
import numpy as np

#### Load csv

In [ ]:
KAGGLE_PATH = '/kaggle/input/fullGas_clean/fullGas_clean.csv'
LOCAL_PATH = '../data/fullGas.csv' 

if os.path.exists(KAGGLE_PATH):
    df = pd.read_csv(KAGGLE_PATH)
    print("Loading files in Kaggle...")
else:
    df = pd.read_csv(LOCAL_PATH)
    print("Loading local files...")

Loading local files...


#### Check data types in variables

In [27]:
df.dtypes

Make                     object
Model                    object
Body                     object
Mileage_km                int64
Price                     int64
Year                    float64
Country                  object
Condition                object
Fuel_Type                object
Fuel_Consumption_l      float64
Drivetrain               object
Gearbox                  object
Gears                   float64
Power_hp                  int64
Engine_Size_cc          float64
Cylinders               float64
Seats                   float64
Doors                   float64
Color                    object
Upholstery               object
Full_Service_History       bool
Non_Smoker_Vehicle         bool
Previous_Owners         float64
Seller                   object
Image_url                object
dtype: object

##### Convert float variables into nullable ints:

In [28]:
cols = [
    'Engine_Size_cc', 'Year', 'Gears',
    'Cylinders', 'Seats', 'Doors', 'Previous_Owners'
]

df[cols] = df[cols].astype('Int64')

Filter out rows containing cars from model years older than 1980, since they're out of scope for this project, and missing many variables.

In [29]:
df = df.loc[df["Year"] >= 1980]

### Data Cleaning
Check for missing values.

In [30]:
for col in df:
    nan_ratio = (df[col].isna().sum() / len(df))*100
    print(col,':', f"{nan_ratio:.1f}%")

Make : 0.0%
Model : 0.3%
Body : 0.0%
Mileage_km : 0.0%
Price : 0.0%
Year : 0.0%
Country : 6.4%
Condition : 0.0%
Fuel_Type : 0.1%
Fuel_Consumption_l : 67.0%
Drivetrain : 21.1%
Gearbox : 1.0%
Gears : 37.6%
Power_hp : 0.0%
Engine_Size_cc : 13.9%
Cylinders : 24.2%
Seats : 4.1%
Doors : 2.1%
Color : 6.2%
Upholstery : 18.3%
Full_Service_History : 0.0%
Non_Smoker_Vehicle : 0.0%
Previous_Owners : 43.0%
Seller : 0.4%
Image_url : 0.0%


Drop columns with more than 30% of missing values

In [31]:
for col in df:
    nan_ratio = (df[col].isna().sum() / len(df))*100
    if nan_ratio > 30:
        df = df.drop(col, axis=1)
    print(col,':', f"{nan_ratio:.1f}%")

Make : 0.0%
Model : 0.3%
Body : 0.0%
Mileage_km : 0.0%
Price : 0.0%
Year : 0.0%
Country : 6.4%
Condition : 0.0%
Fuel_Type : 0.1%
Fuel_Consumption_l : 67.0%
Drivetrain : 21.1%
Gearbox : 1.0%
Gears : 37.6%
Power_hp : 0.0%
Engine_Size_cc : 13.9%
Cylinders : 24.2%
Seats : 4.1%
Doors : 2.1%
Color : 6.2%
Upholstery : 18.3%
Full_Service_History : 0.0%
Non_Smoker_Vehicle : 0.0%
Previous_Owners : 43.0%
Seller : 0.4%
Image_url : 0.0%


### Conditional imputation for electric vehicles

For electric vehicles, certain mechanical attributes such as engine displacement, number of cylinders, and gearbox configuration are not applicable.

Instead of imputing these missing values statistically, they were replaced with explicit “not_applicable” indicators to avoid introducing misleading information.

In [32]:
electric_mask = df["Fuel_Type"].str.lower() == "electric"

In [33]:
df.loc[electric_mask, "Engine_Size_cc"] = df.loc[electric_mask, "Engine_Size_cc"].fillna(0)
df.loc[electric_mask, "Cylinders"] = df.loc[electric_mask, "Cylinders"].fillna(0)
df.loc[electric_mask, "Gearbox"] = df.loc[electric_mask, "Gearbox"].fillna("not_applicable")

#### Transform NaN values to "not reported" for non-relevant categorical variables with a high proportion of missing data. 
I chose this approach instead of imputing missing values with the mode, as doing so would distort the dataset and introduce bias.

In [34]:
df["Drivetrain"] = df["Drivetrain"].fillna("not_reported")
df["Upholstery"] = df["Upholstery"].fillna("not_reported")
df["Country"] = df["Country"].fillna("not_reported")
df["Gearbox"] = df["Gearbox"].fillna("not_reported")

##### Create a function to check for duplicates (e.g. Front Wheel Drive/FWD)
This function was used on all variables, but for practical purposes only one variable is shown here as an example.

In [35]:
def check_unique_values(df, column):
    return sorted(df[column].dropna().unique())

check_unique_values(df, "Drivetrain")

['4WD', 'Front Wheel Drive', 'Rear Wheel Drive', 'not_reported']

Normalization for 'Drivetrain' variable.

In [36]:
df['Drivetrain'] = df['Drivetrain'].replace({'Front Wheel Drive': 'FWD', 'Rear Wheel Drive': 'RWD'})
df['Drivetrain'].unique()

array(['FWD', 'not_reported', 'RWD', '4WD'], dtype=object)

In [37]:
df = df.dropna(subset=['Model'])

##### Detecting outliers on numeric variables using Z-score:
##### Price


The Z-score shows how many standard deviations a data point is from the mean. 
This score is calculated considering each brand's average selling price on their cars, since using the global mean wouldn't be representative.

In [38]:
def z_score(series):
    return (series - series.mean()) / series.std()

def z_score_by_group(df, value_col, group_col):
    return df.groupby(group_col)[value_col].transform(z_score)

We have many outliers, but almost every one of them is justifiable, except for the 9999999USD 2009 C3. Probably a scraping bug.

In [39]:
z_brand = z_score_by_group(df, value_col='Price', group_col='Make')
price_outliers = np.abs(z_brand) > 3
df.loc[price_outliers].sort_values(by=['Price', 'Make'], ascending=[False, False])

,Make,Model,Body,Mileage_km,Price,Year,Country,Condition,Fuel_Type,Drivetrain,...,Engine_Size_cc,Cylinders,Seats,Doors,Color,Upholstery,Full_Service_History,Non_Smoker_Vehicle,Seller,Image_url
8280,Citroen,C3,Sedan,85516,9999999,2009,BE,Used,Gasoline,not_reported,...,1360,4,5,5,Blue,Cloth,False,False,Dealer,https://prod.pictures.autoscout24.net/listing-...
12156,Ferrari,Enzo Ferrari,Coupe,14900,7500000,2004,DE,Used,Gasoline,RWD,...,5998,12,2,2,Yellow,Full leather,True,True,Dealer,https://prod.pictures.autoscout24.net/listing-...
12203,Ferrari,F50,Other,32000,6500000,1996,FR,Used,Gasoline,RWD,...,4700,12,2,2,Yellow,not_reported,False,False,Dealer,https://prod.pictures.autoscout24.net/listing-...
12464,Ferrari,288,Other,1,6500000,1985,ES,Used,Gasoline,not_reported,...,<NA>,<NA>,<NA>,5,Red,not_reported,False,False,Dealer,https://prod.pictures.autoscout24.net/listing-...
12064,Ferrari,Enzo Ferrari,Coupe,13222,6400000,2003,FR,Used,Gasoline,not_reported,...,5998,12,2,2,Yellow,Other,False,False,Dealer,https://prod.pictures.autoscout24.net/listing-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10241,Dacia,Bigster,Off-Road/Pick-up,3122,33990,2025,DE,Used,Electric/Gasoline,FWD,...,1799,4,5,5,Brown,Cloth,False,False,Dealer,https://prod.pictures.autoscout24.net/listing-...
30001,Opel,Insignia,Station wagon,27294,33950,2022,DE,Used,Gasoline,FWD,...,1998,4,5,5,Grey,Full leather,False,True,Dealer,https://prod.pictures.autoscout24.net/listing-...
10432,Dacia,Bigster,Off-Road/Pick-up,1500,33900,2025,DE,Used,Electric/Gasoline,FWD,...,1799,4,5,5,Black,Other,True,True,Dealer,https://prod.pictures.autoscout24.net/listing-...
13330,Fiat,Doblo,Van,10,32900,2025,IT,Used,Diesel,FWD,...,1499,4,5,5,Blue,Cloth,False,False,Dealer,https://prod.pictures.autoscout24.net/listing-...


Drop the 10 million dollar Citroen.

In [40]:
df = df.drop(index=8280)

#### Mileage

In [41]:
z_mileage = z_score(df['Mileage_km'])
mileage_outliers = np.abs(z_mileage) > 3
df.loc[mileage_outliers].sort_values(by=['Mileage_km'], ascending=[False])

,Make,Model,Body,Mileage_km,Price,Year,Country,Condition,Fuel_Type,Drivetrain,...,Engine_Size_cc,Cylinders,Seats,Doors,Color,Upholstery,Full_Service_History,Non_Smoker_Vehicle,Seller,Image_url
414,Abarth,595 Turismo,Other,16777215,8500,2013,NL,Used,Gasoline,not_reported,...,<NA>,<NA>,<NA>,<NA>,Black,not_reported,False,False,Dealer,https://prod.pictures.autoscout24.net/listing-...
3854,Bentley,Brooklands,Other,16777215,5500,1993,NL,Used,Gasoline,not_reported,...,<NA>,<NA>,<NA>,<NA>,Black,not_reported,False,False,Dealer,https://prod.pictures.autoscout24.net/listing-...
3881,Bentley,Continental GT,Other,16777215,19500,2004,NL,Used,Gasoline,not_reported,...,<NA>,<NA>,<NA>,<NA>,Black,not_reported,False,False,Dealer,https://prod.pictures.autoscout24.net/listing-...
3956,Bentley,Continental GTC,Other,16777215,29500,2008,NL,Used,Gasoline,not_reported,...,<NA>,<NA>,<NA>,<NA>,Black,not_reported,False,False,Dealer,https://prod.pictures.autoscout24.net/listing-...
7073,Chevrolet,Camaro,Other,16777215,6500,1985,NL,Used,Gasoline,not_reported,...,<NA>,<NA>,<NA>,<NA>,Black,not_reported,False,False,Dealer,https://prod.pictures.autoscout24.net/listing-...
7158,Chevrolet,Camaro,Other,16777215,4500,1997,NL,Used,Gasoline,not_reported,...,<NA>,<NA>,<NA>,<NA>,Black,not_reported,False,False,Dealer,https://prod.pictures.autoscout24.net/listing-...
20640,Lancia,Delta,Other,16777215,22500,1988,NL,Used,Gasoline,not_reported,...,<NA>,<NA>,<NA>,<NA>,Black,not_reported,False,False,Dealer,https://prod.pictures.autoscout24.net/listing-...
23382,Maserati,Quattroporte,Other,16777215,20500,2016,NL,Used,Gasoline,not_reported,...,<NA>,<NA>,<NA>,<NA>,Black,not_reported,False,False,Dealer,https://prod.pictures.autoscout24.net/listing-...
32963,Rolls-Royce,Silver Seraph,Other,16777215,38500,1999,NL,Used,Gasoline,not_reported,...,<NA>,<NA>,<NA>,<NA>,Black,not_reported,False,False,Dealer,https://prod.pictures.autoscout24.net/listing-...
35977,smart,forTwo,Compact,9999999,4490,2009,IT,Used,Gasoline,RWD,...,999,3,2,2,NaN,not_reported,False,True,Dealer,https://prod.pictures.autoscout24.net/listing-...


Rows with a mileage higher than 9999999 are dropped, since all of them are incorrect.

In [42]:
df = df.loc[df["Mileage_km"] < 9999999]

Export the clean data to a csv file to continue with the EDA in other platform.

In [43]:
df.to_csv("../data/fullGas_clean.csv", index=False)
%store df 

Stored 'df' (DataFrame)
